In [1]:
import os

# Replace with your actual EOG credentials
os.environ["EOG_USERNAME"] = ""
os.environ["EOG_PASSWORD"] = ""

import os
import requests
from pathlib import Path
from bs4 import BeautifulSoup
from getpass import getpass

def get_access_token(username, password):
    try:
        token_url = "https://eogauth-new.mines.edu/realms/eog/protocol/openid-connect/token"
        params = {
            "client_id": "",
            "client_secret": "",
            "username": username,
            "password": password,
            "grant_type": "password"
        }
        response = requests.post(token_url, data=params)
        response.raise_for_status()
        return response.json().get("access_token")
    except Exception as e:
        print(f"❌ Failed to get access token: {e}")
        return None

In [ ]:
!pip install rasterio

from pathlib import Path
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.warp import reproject, Resampling
from rasterio.features import geometry_mask
from shapely.ops import transform
from functools import partial
import pyproj
import requests
from bs4 import BeautifulSoup

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



def download_annual_viirs_files(base_url, year, output_dir, access_token):
    """
    Downloads average.dat.tif.gz and cf_cvg.dat.tif.gz for a given year using EOG OAuth2 token.
    """
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    year_url = f"{base_url}/{year}/"
    response = requests.get(year_url, headers=headers, verify=False)
    if response.status_code != 200:
        print(f"❌ Failed to access {year_url}")
        return

    soup = BeautifulSoup(response.text, "html.parser")
    links = [a["href"] for a in soup.find_all("a") if a.get("href")]

    for keyword in ["average.dat.tif.gz", "cf_cvg.dat.tif.gz"]:
        for link in links:
            if keyword in link:
                file_url = f"{year_url}{link}"
                file_name = link.split("/")[-1]
                output_path = output_dir / file_name

                if output_path.exists():
                    print(f"⏩ Already downloaded: {file_name}")
                    continue

                print(f"⬇️ Downloading: {file_name}")
                with requests.get(file_url, headers=headers, stream=True, verify=False) as r:
                    r.raise_for_status()
                    with open(output_path, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                print(f"✅ Saved to: {output_path}")


country = "id"
year = "2018"

def extract_and_process_annual_ntl(archive_dir, output_dir, zones_path, mask_path, field_id="ADM2_PCODE"):
    """
    Processes annual NOAA NTL composites:
    - Uses pre-extracted average.dat.tif.gz and cf_cvg.dat.tif.gz
    - Applies cf_cvg > 0 mask
    - Applies external ephemeral mask
    - Computes zonal statistics
    """

    archive_dir = Path(archive_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    tif_pairs = {}

    # for gz in archive_dir.glob("*.tif.gz"):
    #     if "average.dat.tif.gz" in gz.name:
    #         year = gz.name.split("_")[3]  # e.g., '2021'
    #         tif_pairs.setdefault(year, {})["avg"] = gz
    #     elif "cf_cvg.dat.tif.gz" in gz.name:
    #         year = gz.name.split("_")[3]
    #         tif_pairs.setdefault(year, {})["cvg"] = gz

    # years_to_include = ['2020', '2021', '2022']
    years_to_include = ['2018']

    # for gz in archive_dir.glob("*.tif.gz"):
    #     parts = gz.name.split("_")
    #     if len(parts) > 3:
    #         year = parts[3]
    #         if year not in years_to_include:
    #             continue
    #         if "average.dat.tif.gz" in gz.name:
    #             tif_pairs.setdefault(year, {})["avg"] = gz
    #         elif "cf_cvg.dat.tif.gz" in gz.name:
    #             tif_pairs.setdefault(year, {})["cvg"] = gz

    for tif in archive_dir.glob("*.tif"):
        if f"{country}_{year}_avg_m.tif" in tif.name:
            print("ok")
            tif_pairs.setdefault("manual", {})["avg"] = tif
        elif f"{country}_{year}c.tif" in tif.name:
            tif_pairs.setdefault("manual", {})["cvg"] = tif

    for year, files in tif_pairs.items():
        print(f"\n🌀 Processing year: {year}")

        if not ("avg" in files and "cvg" in files):
            print(f"❌ Missing average or cf_cvg file for {year}")
            continue

        avg_path = output_dir / f"{year}_avg.tif"
        cvg_path = output_dir / f"{year}_cf_cvg.tif"

        if not avg_path.exists():
            with rasterio.open(f"gzip://{files['avg']}") as src:
                with rasterio.open(avg_path, "w", **src.profile) as dst:
                    dst.write(src.read())

        if not cvg_path.exists():
            with rasterio.open(f"gzip://{files['cvg']}") as src:
                with rasterio.open(cvg_path, "w", **src.profile) as dst:
                    dst.write(src.read())

        with rasterio.open(avg_path) as src_avg, rasterio.open(cvg_path) as src_cvg:
            avg = src_avg.read(1).astype(np.float32)
            cvg = src_cvg.read(1)
            masked = np.where(cvg > 0, avg, 0)
            profile = src_avg.profile
            profile.update(dtype='float32')
            masked_path = output_dir / f"{year}_masked_avg.tif"
            with rasterio.open(masked_path, "w", **profile) as dst:
                dst.write(masked, 1)

        with rasterio.open(masked_path) as ref:
            with rasterio.open(mask_path) as mask_src:
                mask_data = mask_src.read(1)
                resampled = np.empty((ref.height, ref.width), dtype=mask_data.dtype)
                reproject(
                    source=mask_data,
                    destination=resampled,
                    src_transform=mask_src.transform,
                    src_crs=mask_src.crs,
                    dst_transform=ref.transform,
                    dst_crs=ref.crs,
                    resampling=Resampling.nearest
                )
                resampled_mask_path = output_dir / f"{year}_resampled_mask.tif"
                mask_profile = ref.profile.copy()
                mask_profile.update(dtype=mask_data.dtype)
                with rasterio.open(resampled_mask_path, "w", **mask_profile) as dst:
                    dst.write(resampled, 1)

        with rasterio.open(masked_path) as src_masked, rasterio.open(resampled_mask_path) as mask_src:
            data = src_masked.read(1).astype(np.float32)
            mask = mask_src.read(1)
            final = np.where(mask == 1, data, 0)
            final_path = output_dir / f"{year}_final_masked.tif"
            with rasterio.open(final_path, "w", **src_masked.profile) as dst:
                dst.write(final, 1)

        print(f"🧮 Computing zonal stats for {year}")
        gdf = gpd.read_file(zones_path)
        with rasterio.open(final_path) as src:
            if gdf.crs != src.crs:
                gdf = gdf.to_crs(src.crs)
            try:
                proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'), pyproj.Proj(init='esri:102025'))
            except:
                proj = partial(pyproj.transform, pyproj.Proj('epsg:4326'), pyproj.Proj('esri:102025'))
            gdf["area_km2"] = [transform(proj, geom).area / 1e6 for geom in gdf.geometry]

            array = src.read(1)
            nodata = src.nodata
            array[array == nodata] = np.nan

            count_lit = []
            for geom in gdf.geometry:
                mask = geometry_mask([geom], transform=src.transform, invert=True,
                                     out_shape=(src.height, src.width))
                count_lit.append(np.sum(array[mask] > 0))

        gdf["count_lit"] = count_lit
        gdf[[field_id, "ADM2_EN", "ADM1_EN", "ADM1_PCODE", "area_km2", "count_lit"]].to_csv(output_dir / f"{year}_zonal.csv", index=False)
        print(f"✅ Finished: {year}_zonal.csv")


extract_and_process_annual_ntl(
    archive_dir="ntl_unzipped_annual",
    output_dir="processed_annual",
    # zones_path="th1.shp",  # Replace with your actual shapefile name
    zones_path="idn_admbnda_adm2_bps_20200401.shp",
    # zones_path="ph3_v2.shp",# Replace with your actual shapefile name
    mask_path = "id_2018.tif",
    field_id="ADM2_PCODE"
)

print('yes')

UnboundLocalError: cannot access local variable 'year' where it is not associated with a value

In [ ]:
## with dropping eph lights (base)
# !pip install rasterio
# !pip install exactextract

from pathlib import Path
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.ops import transform
from functools import partial
import pyproj
from exactextract import exact_extract

from pathlib import Path
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.warp import reproject, Resampling
from rasterio.features import geometry_mask
from shapely.ops import transform
from functools import partial
import pyproj
import requests
from bs4 import BeautifulSoup

country = "id"
year = "2017"

def extract_and_process_annual_ntl(archive_dir, output_dir, zones_path, mask_path, field_id="ADM2_PCODE", country=None, year=None):
    """
    Processes annual NOAA NTL composites:
    - Uses pre-extracted average.dat.tif.gz and cf_cvg.dat.tif.gz
    - Applies cf_cvg > 0 mask
    - Applies external ephemeral mask
    - Computes zonal statistics
    """

    archive_dir = Path(archive_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    tif_pairs = {}

    # Scan for pre-named TIFFs (manual extraction case)
    for tif in archive_dir.glob("*.tif"):
        if f"{country}_{year}_avg_m.tif" in tif.name:
            print("ok")
            tif_pairs.setdefault("manual", {})["avg"] = tif
        elif f"{country}_{year}c.tif" in tif.name:
            tif_pairs.setdefault("manual", {})["cvg"] = tif

    for yr, files in tif_pairs.items():
        print(f"\n🌀 Processing year: {yr}")

        if not ("avg" in files and "cvg" in files):
            print(f"❌ Missing average or cf_cvg file for {yr}")
            continue

        avg_path = output_dir / f"{yr}_avg.tif"
        cvg_path = output_dir / f"{yr}_cf_cvg.tif"

        # No gzip needed here because input is already plain .tif
        if not avg_path.exists():
            with rasterio.open(files["avg"]) as src:
                with rasterio.open(avg_path, "w", **src.profile) as dst:
                    dst.write(src.read())

        if not cvg_path.exists():
            with rasterio.open(files["cvg"]) as src:
                with rasterio.open(cvg_path, "w", **src.profile) as dst:
                    dst.write(src.read())

        with rasterio.open(avg_path) as src_avg, rasterio.open(cvg_path) as src_cvg:
            avg = src_avg.read(1).astype(np.float32)
            cvg = src_cvg.read(1)
            masked = np.where(cvg > 0, avg, 0)
            profile = src_avg.profile
            profile.update(dtype="float32")
            masked_path = output_dir / f"{yr}_masked_avg.tif"
            with rasterio.open(masked_path, "w", **profile) as dst:
                dst.write(masked, 1)

        with rasterio.open(masked_path) as ref:
            with rasterio.open(mask_path) as mask_src:
                mask_data = mask_src.read(1)
                resampled = np.empty((ref.height, ref.width), dtype=mask_data.dtype)
                reproject(
                    source=mask_data,
                    destination=resampled,
                    src_transform=mask_src.transform,
                    src_crs=mask_src.crs,
                    dst_transform=ref.transform,
                    dst_crs=ref.crs,
                    resampling=Resampling.nearest,
                )
                resampled_mask_path = output_dir / f"{yr}_resampled_mask.tif"
                mask_profile = ref.profile.copy()
                mask_profile.update(dtype=mask_data.dtype)
                with rasterio.open(resampled_mask_path, "w", **mask_profile) as dst:
                    dst.write(resampled, 1)

        with rasterio.open(masked_path) as src_masked, rasterio.open(resampled_mask_path) as mask_src:
            data = src_masked.read(1).astype(np.float32)
            mask = mask_src.read(1)
            final = np.where(mask == 1, data, 0)
            final_path = output_dir / f"{yr}_final_masked.tif"
            with rasterio.open(final_path, "w", **src_masked.profile) as dst:
                dst.write(final, 1)

        print(f"🧮 Computing zonal stats for {yr}")
        gdf = gpd.read_file(zones_path)
        with rasterio.open(final_path) as src:
              if gdf.crs != src.crs:
                  gdf = gdf.to_crs(src.crs)

              # Run exact_extract with multiple stats
              stats = exact_extract(
                  src, gdf,
                  {
                      "count": "count",
                      "sum": "sum",
                      "mean": "mean",
                      "stdev": "stdev",
                      "min": "min",
                      "max": "max",
                      "median": "median"
                  }
              )

              # Helper function to safely extract from results
              def extract_stat(stats_list, stat_name):
                  return [
                      s["properties"].get(stat_name, None)
                      if isinstance(s, dict) and "properties" in s else None
                      for s in stats_list
                  ]

              # Build dataframe with selected attributes
              df = pd.DataFrame({
                  field_id: gdf[field_id],
                  "ADM1_PCODE": gdf["ADM1_PCODE"],
                  "ADM1_EN": gdf["ADM1_EN"],
                  "min": extract_stat(stats, "min"),
                  "max": extract_stat(stats, "max"),
                  "mean": extract_stat(stats, "mean"),
                  "count": extract_stat(stats, "count"),
                  "sum": extract_stat(stats, "sum"),
                  "std": extract_stat(stats, "stdev"),
                  "median": extract_stat(stats, "median")
              })

              # Also compute count_lit (pixels > 0)
              with rasterio.open(final_path) as src:
                  array = src.read(1).astype(float)
                  nodata = src.nodata
                  array[array == nodata] = np.nan

                  count_lit = []
                  for geom in gdf.geometry:
                      mask = rasterio.features.geometry_mask(
                          [geom], transform=src.transform, invert=True,
                          out_shape=(src.height, src.width)
                      )
                      values = array[mask]
                      count_lit.append(int(np.ceil(np.sum(values > 0))))

              df["count_lit"] = count_lit

              # Save to CSV
              out_csv = output_dir / f"{country}_{year}_prov.csv"
              df[[field_id, "ADM2_PCODE", "ADM1_EN", "ADM1_PCODE", "count_lit", "mean", "min", "max", "std", "sum", "median"]].to_csv(out_csv, index=False)
              print(f"✅ Zonal stats saved to: {out_csv}")


# Run function with explicit args
extract_and_process_annual_ntl(
    archive_dir="ntl_unzipped_annual",
    output_dir="processed_annual",
    zones_path="idn_admbnda_adm2_bps_20200401.shp",
    mask_path="id_2017.tif",
    field_id="ADM2_PCODE",
    country=country,
    year=year,
)

ok

🌀 Processing year: manual
🧮 Computing zonal stats for manual
✅ Zonal stats saved to: processed_annual/id_2017_prov.csv


In [ ]:
#### manual; without dropping eph lights (base)

# !pip install rasterio
# !pip install exactextract

from pathlib import Path
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.ops import transform
from functools import partial
import pyproj
from exactextract import exact_extract


country = "ph"
year = "2021"

def extract_and_process_annual_ntl(archive_dir, output_dir, zones_path, field_id="ADM2_PCODE"):
    """
    Processes manually uploaded average_masked.dat.tif and cf_cvg.dat.tif files:
    - Applies cf_cvg > 0 mask
    - Computes zonal statistics
    """

    archive_dir = Path(archive_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    tif_pairs = {}
    for tif in archive_dir.glob("*.tif"):
        if f"{country}_{year}_avg_m.tif" in tif.name:
            print('ok')
            tif_pairs.setdefault("manual", {})["avg"] = tif
        elif f"{country}_{year}_cf_cvg.tif" in tif.name:
            tif_pairs.setdefault("manual", {})["cvg"] = tif

    # for tif in archive_dir.glob("*.tif"):
    #     if "average.dat.tif" in tif.name:
    #         tif_pairs.setdefault("manual", {})["avg"] = tif
    #     elif "cf_cvg.dat.tif" in tif.name:
    #         tif_pairs.setdefault("manual", {})["cvg"] = tif

    for label, files in tif_pairs.items():
        print(f"\n🌀 Processing uploaded set")

        if not ("avg" in files and "cvg" in files):
            print(f"❌ Missing average_masked or cf_cvg file")
            continue

        avg_path = files['avg']
        cvg_path = files['cvg']

        with rasterio.open(avg_path) as src_avg, rasterio.open(cvg_path) as src_cvg:
            avg = src_avg.read(1).astype(np.float32)
            cvg = src_cvg.read(1)
            masked = np.where(cvg >= 0, avg, 0)
            profile = src_avg.profile
            profile.update(dtype='float32')
            final_path = output_dir / "final_masked.tif"
            with rasterio.open(final_path, "w", **profile) as dst:
                dst.write(masked, 1)

        print(f"🧮 Computing zonal stats")
        gdf = gpd.read_file(zones_path)
        with rasterio.open(final_path) as src:
            if gdf.crs != src.crs:
                gdf = gdf.to_crs(src.crs)
            try:
                proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'), pyproj.Proj(init='esri:102025'))
            except:
                proj = partial(pyproj.transform, pyproj.Proj('epsg:4326'), pyproj.Proj('esri:102025'))

            # stats = exact_extract(src, gdf, ["count", "sum", "mean", "stdev", "min", "max", "median"])
            stats = exact_extract(src, gdf, {
                "count": "count",
                "sum": "sum",
                "mean": "mean",
                "stdev": "stdev",
                "min": "min",
                "max": "max",
                "median": "median"
            })

            # Extract statistics from 'properties' key
        def extract_stat(stats_list, stat_name):
            """ Extracts a specific statistic from the 'properties' field safely. """
            return [s["properties"].get(stat_name, None) if isinstance(s, dict) and "properties" in s else None for s in stats_list]


        df = pd.DataFrame({
                field_id: gdf[field_id],
                "ADM3_EN": gdf["ADM3_EN"],
                "ADM2_PCODE": gdf["ADM2_PCODE"],
                "ADM2_EN": gdf["ADM2_EN"],
                "ADM1_PCODE": gdf["ADM1_PCODE"],
                "ADM1_EN": gdf["ADM1_EN"],
                # "area_km2": gdf["area_km2"],
                "min": extract_stat(stats, "min"),
                "max": extract_stat(stats, "max"),
                "mean": extract_stat(stats, "mean"),
                "count": extract_stat(stats, "count"),
                "sum": extract_stat(stats, "sum"),
                "std": extract_stat(stats, "stdev"),
                "median": extract_stat(stats, "median")
            })

        # Also calculate count_lit (pixels > 0)
        with rasterio.open(final_path) as src:
            array = src.read(1).astype(float)
            nodata = src.nodata
            array[array == nodata] = np.nan

            count_lit = []
            for geom in gdf.geometry:
                mask = rasterio.features.geometry_mask([geom], transform=src.transform, invert=True, out_shape=(src.height, src.width))
                values = array[mask]
                count_lit.append(int(np.ceil(np.sum(values > 0))))

        df["count_lit"] = count_lit

        df[[field_id, "ADM3_EN", "ADM2_PCODE", "ADM2_EN", "ADM1_PCODE", "ADM1_EN", "count_lit", "mean", "min", "max", "std", "sum", "median"]].to_csv(output_dir / f"{country}_{year}.csv", index=False)

        # df[[field_id, "ADM2_EN", "ADM1_PCODE","ADM1_EN", "count_lit", "mean", "min", "max", "std", "sum", "median"]].to_csv(output_dir / f"{country}_{year}_reg_cvg0_2.csv", index=False)
        # df[[field_id, "ADM1_EN", "count_lit", "mean", "min", "max", "std", "sum", "median"]].to_csv(output_dir / f"{country}_{year}_prov_cvg0_2.csv", index=False)

        print(f"✅ Zonal stats saved to: {output_dir}/zonal_stats.csv")


extract_and_process_annual_ntl(
    archive_dir="ntl_unzipped_annual",
    output_dir="processed_annual",
    # zones_path="th1.shp",  # Replace with your actual shapefile name
    # zones_path="idn_admbnda_adm2_bps_20200401.shp",
    # zones_path="id1.shp",
    zones_path="ph3_v2.shp",# Replace with your actual shapefile name
    field_id="ADM3_PCODE"
)

print('yes')

ok

🌀 Processing uploaded set
🧮 Computing zonal stats


/usr/local/lib/python3.12/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.12/dist-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


✅ Zonal stats saved to: processed_annual/zonal_stats.csv
yes
